In [ ]:
Transfer Learning Homework 

Below you see a tutorial from Keras on using transfer learning. They train their models on half  the MNIST dataset digits (i.e. digits 0-4) and “transfer” the model to the second half (i.e, digits 5-9). 

Your homework is to train on all digits and make your own handwritten data set of the 5 characters {A, B, C, D, E}  and “transfer” your MNIST trained model over to the dataset you created. In other words, train on the full MNIST datasets (i.e. digits 0-9) and transfer on the {A, B, C, D, E} image dataset. 

Please do not use any other data resources from the web such as the emnist dataset. Figuring out the challenges of making your own handwritten character dataset is part of the exercise for this homework!   

Include the images you generated as part of your submission. Include any code used for the character image preprocessing. In a README file include details on your data generation process.

Code source: https://github.com/awslabs/keras-apache-mxnet/blob/master/examples/mnist_transfer_cnn.py


In [37]:
from __future__ import print_function
import datetime
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import os
from keras.utils import img_to_array, load_img
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

now = datetime.datetime.now
batch_size = 128
num_classes = 10
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

(x_train, y_train), (x_test, y_test) = mnist.load_data()

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model,
            (x_train, y_train),
            (x_test, y_test), num_classes)

# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

folder = '../letters HW4/'
for f in os.listdir(folder):
     if f.endswith('.png'):
        img = load_img(os.path.join(folder, f), color_mode='grayscale', target_size=(28, 28))
        x = img_to_array(img)
        x /= 255
        preds = model.predict(np.array([x]))
    #    decoded_preds = decode_predictions(preds, top=3)[0]
        print('Predictions for', preds)
    #    for pred in decoded_preds:
    #        print(pred[1], ':', pred[2])



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/5


2023-03-05 15:12:19.888044: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - ETA: 0s - loss: 2.2983 - accuracy: 0.1267

2023-03-05 15:12:28.816005: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 10s 20ms/step - loss: 2.2983 - accuracy: 0.1267 - val_loss: 2.2649 - val_accuracy: 0.2423
Epoch 2/5
469/469 [==============================] - 9s 19ms/step - loss: 2.2523 - accuracy: 0.1935 - val_loss: 2.2148 - val_accuracy: 0.4235
Epoch 3/5
469/469 [==============================] - 9s 20ms/step - loss: 2.2032 - accuracy: 0.2655 - val_loss: 2.1536 - val_accuracy: 0.5519
Epoch 4/5
469/469 [==============================] - 9s 19ms/step - loss: 2.1397 - accuracy: 0.3374 - val_loss: 2.0727 - val_accuracy: 0.6289
Epoch 5/5
469/469 [==============================] - 9s 19ms/step - loss: 2.0577 - accuracy: 0.4031 - val_loss: 1.9668 - val_accuracy: 0.6790
Training time: 0:00:46.375883
Test score: 1.966835856437683
Test accuracy: 0.6790000200271606
1/1 [==============================] - 0s 78ms/step
Predictions for [[0.19075683 0.03311579 0.22793788 0.15051094 0.05100118 0.06205777
  0.05684206 0.05913936 0.12617029 0.0424679 ]]
1/1 [================

2023-03-05 15:13:08.899793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Predictions for [[0.18896683 0.03467231 0.23155785 0.14778133 0.05130074 0.06215703
  0.05591715 0.05764651 0.12735257 0.04264766]]
